<a href="https://colab.research.google.com/github/JaminUbuntu/JaminUbuntu/blob/main/LinearRegression_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U scikit-learn


In [ ]:
pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np
%matplotlib inline

In [ ]:
df = pd.read_csv("/content/Ecommerce_Customers (1).csv")
df.head(3)

,Email,Address,Avatar,Avg Session Length,Time on App,Time on Website,Length of Membership,Yearly Amount Spent
0,mstephenson@fernandez.com,"835 Frank TunnelWrightmouth, MI 82180-9605",Violet,34.497268,12.655651,39.577668,4.082621,587.951054
1,hduke@hotmail.com,"4547 Archer CommonDiazchester, CA 06566-8576",DarkGreen,31.926272,11.109461,37.268959,2.664034,392.204933
2,pallen@yahoo.com,"24645 Valerie Unions Suite 582Cobbborough, DC ...",Bisque,33.000915,11.330278,37.110597,4.104543,487.547505


In [ ]:
df.columns

Index(['Email', 'Address', 'Avatar', 'Avg Session Length', 'Time on App',
       'Time on Website', 'Length of Membership', 'Yearly Amount Spent'],
      dtype='object')

Linear Regression with Pyspark


*   First you start a spark session




In [ ]:
pip install findspark


In [ ]:
import findspark

In [ ]:
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('lr_test_session').getOrCreate()

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
data = spark.read.csv("/content/Ecommerce_Customers (1).csv", inferSchema=True, header=True)

In [ ]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [ ]:
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App",
               "Time on Website", "Length of Membership"],
    outputCol="features")

In [ ]:
output = assembler.transform(data)

In [ ]:
output.select("features").show(3)

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
+--------------------+
only showing top 3 rows



In [ ]:
output.show(3)

+--------------------+--------------------+---------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|   Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+---------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|   Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|   Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487

In [ ]:
final_data = output.select("features", "Yearly Amount Spent")

In [ ]:
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [ ]:
final_data.show(3)

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
+--------------------+-------------------+
only showing top 3 rows



In [ ]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [ ]:
train_data.show(3)

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[30.3931845423455...|  319.9288698031936|
|[30.4925366965402...|  282.4712457199145|
|[30.8162006488763...|   266.086340948469|
+--------------------+-------------------+
only showing top 3 rows



In [ ]:
test_data.show(3)

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[29.5324289670579...|  408.6403510726275|
|[30.5743636841713...| 442.06441375806565|
|[30.7377203726281...|  461.7807421962299|
+--------------------+-------------------+
only showing top 3 rows



In [ ]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                333|
|   mean|  497.0601198671003|
| stddev|  80.58083163419772|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



In [ ]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                167|
|   mean|  503.8083785242575|
| stddev|  76.76814301307512|
|    min| 256.67058229005585|
|    max|  700.9170916173961|
+-------+-------------------+



In [ ]:
lr = LinearRegression(labelCol="Yearly Amount Spent")

In [ ]:
lrModel = lr.fit(train_data)

In [ ]:
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [25.794352574090013,38.863587470957036,0.6351649741569511,61.760311709711566] Intercept: -1063.4502365969934


In [ ]:
test_results = lrModel.evaluate(test_data)

In [ ]:
test_results.residuals.show(3)

+-------------------+
|          residuals|
+-------------------+
| 10.648957159210795|
|0.29113791171010917|
| 10.884155984852782|
+-------------------+
only showing top 3 rows



In [ ]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))

RMSE: 9.55725552975766


Now that we have built our model
Its time to test the model with unlabeled data

In [ ]:
unlabeled_data = test_data.select("features")

# Prediction

In [ ]:
predictions = lrModel.transform(unlabeled_data)

In [ ]:
predictions.show(3)

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 397.9913939134167|
|[30.5743636841713...|441.77327584635555|
|[30.7377203726281...| 450.8965862113771|
+--------------------+------------------+
only showing top 3 rows



In [ ]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 9.55725552975766
MSE: 91.34113326108336
